In [1]:
#from gensim import corpora, models
from sklearn.decomposition import LatentDirichletAllocation
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
urbn_soup = BeautifulSoup(open(r"C:\Users\User\Desktop\thesis_17_3_2022\stock_price_and_news\stock_news\dataset\urbn_final\urbn_2003.html", encoding="utf8"))


In [3]:
urbn_timestamp = []
for e in urbn_soup.find_all("span", {"class": "article__timestamp"}):
    urbn_timestamp.append(e.get_text())
urbn_file = []
for gps_text in urbn_soup.find_all("h3"):
    urbn_file.append(gps_text.get_text())
urbn_clean1 = []
for gps_text1 in urbn_file:
    urbn_clean1.append(gps_text1.replace("\n", ""))
urbn_remove = []
for e in urbn_clean1:
    if e not in (
 'No Recent Tickers',
 'Overview'
 ):
        urbn_remove.append(e)
urbn_space = []
for e in urbn_remove:
    urbn_space.append(e.strip())
urbn_fullstop = []
for e in urbn_space:
    urbn_fullstop.append(e.replace(".",""))
urbn_q = []
for e in urbn_fullstop:
    urbn_q.append(e.replace("?",""))
urbn_colon = []
for e in urbn_q:
    urbn_colon.append(e.replace(":",""))
urbn_comma = []
for e in urbn_colon:
    urbn_comma.append(e.replace(",",""))
urbn_percent = []
for e in urbn_comma:
    urbn_percent.append(e.replace("%","percent"))
urbn_lower = []
for i in range(len(urbn_percent)):
    urbn_lower.append(urbn_percent[i].lower())
urbn_barr = []
for e in urbn_lower:
    urbn_barr.append(e.replace("barron's",""))
urbn_pain = []
for e in urbn_barr:
    if e not in ( 'marketwatch',
 'company',
 'dow jones network'):
        urbn_pain.append(e)


In [4]:
urbn_et = []
for e in urbn_timestamp:
    urbn_et.append(e.replace("ET",""))
urbn_at1 = []
for e in urbn_et:
    urbn_at1.append(e.replace("at",""))
urbn_pm1 = []
for e in urbn_at1:
    urbn_pm1.append(e.replace("p.m.",""))
urbn_am1 = []
for e in urbn_pm1:
    urbn_am1.append(e.replace("a.m.",""))
urbn_commas = []
for e in urbn_am1:
    urbn_commas.append(e.replace(",",""))
urbn_fullstops = []
for e in urbn_commas:
    urbn_fullstops.append(e.replace(".",""))
urbn_timestamp_final = urbn_fullstops


In [5]:
urbn_timestamp_final[-5:]

['Feb 22 2022  1:46  ',
 'Feb 18 2022  7:38  ',
 'Feb 15 2022  10:38  ',
 'Feb 13 2022  9:55  ',
 'Feb 11 2022  3:38  ']

In [6]:
urbn_date = [datetime.strptime(x,'%b %d %Y  %I:%M ') for x in urbn_timestamp_final]

In [7]:
tf_vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)

In [8]:
urbn_data = pd.DataFrame(
    {'Date': urbn_date,
     'text_headlines': urbn_pain
     
    })

In [9]:
urbn_data.to_csv('urbn_topic_selection.csv', index = True)

In [10]:
urbn_data.head(3)
urbn_news_matrix = tf_vectorizer.fit_transform(urbn_data['text_headlines'])

In [11]:
import re
urbn_data['news_text_processed'] = urbn_data['text_headlines'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
urbn_data['news_text_processed'] = urbn_data['text_headlines'].map(lambda x: x.lower())
# Print out the first rows of papers
urbn_data['news_text_processed'].head()

0                                                  ...
1                                                  ...
2    urban outfitters shares slip as free people pr...
3    urban outfitters stock slides 39percent premarket
4    urban outfitters says q4 margins hurt by highe...
Name: news_text_processed, dtype: object

In [12]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = urbn_data.news_text_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['retail', 'stocks', 'to', 'avoid', 'amid', 'inflation', 'uncertainty', 'according', 'to', 'barclays']


In [13]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [14]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[['retail', 'stock', 'avoid', 'inflation', 'uncertainty', 'accord', 'barclay']]


In [16]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [17]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [18]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.081*"outfitter" + 0.081*"urban" + 0.036*"stock" + 0.027*"miss" + '
  '0.023*"drop" + 0.022*"sale" + 0.019*"jump" + 0.013*"profit" + 0.011*"say" + '
  '0.010*"result"'),
 (1,
  '0.060*"urban" + 0.060*"outfitter" + 0.036*"sector" + 0.035*"retail" + '
  '0.032*"stock" + 0.029*"upgrade" + 0.016*"lead" + 0.016*"buy" + '
  '0.015*"wal_mart" + 0.012*"neutral"'),
 (2,
  '0.071*"outfitter" + 0.071*"urban" + 0.060*"surprise" + '
  '0.053*"updates_advisorie" + 0.035*"cent" + 0.021*"ep" + 0.016*"percent" + '
  '0.016*"cut" + 0.014*"neutral" + 0.011*"buy"'),
 (3,
  '0.167*"urban" + 0.166*"outfitter" + 0.089*"percent" + 0.069*"share" + '
  '0.054*"sale" + 0.026*"break" + 0.025*"profit" + 0.019*"earning" + '
  '0.019*"fall" + 0.019*"rise"'),
 (4,
  '0.119*"urban" + 0.119*"outfitter" + 0.091*"stock" + 0.082*"price" + '
  '0.081*"target" + 0.054*"cut" + 0.036*"raise" + 0.010*"earning" + '
  '0.010*"mkm_partner" + 0.009*"estimate"'),
 (5,
  '0.042*"retailer" + 0.037*"stock" + 0.021*"outfitter"

In [19]:

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.46293407964310385


In [20]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [21]:
from gensim.models import CoherenceModel
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 6
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

 61%|██████    | 327/540 [1:12:55<25:42,  7.24s/it]  C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))
100%|██████████| 540/540 [2:11:44<00:00, 14.64s/it]
